In [23]:
import numpy as np
import scipy
import combo
import os
import urllib

In [20]:
def load_data():
    ''' Load '''
    download_data()
    A =  np.asarray(np.loadtxt('data/s5-210.csv',skiprows=1,delimiter=','))
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

In [19]:
def download_data():
    ''' Download test data  '''
    if not os.path.exists('data/s5-210.csv'):
        if not os.path.exists('data'):
            os.mkdir('data')
        print('Downloading...')
        urllib.urlretrieve('http://www.tsudalab.org/files/s5-210.csv', 'data/s5-210.csv')
        print('Done')

In [27]:
# Load data for the grain boundary
X, t = load_data()

#  Centering the matrix whose the mean and standard deviation are 0 and 1
X = combo.misc.centering(X)

In [34]:
# Define the class for physical simulator 
class simulator:
    def __init__( self ):
        _, self.t = load_data()
    
    def __call__( self, action ):
        return self.t[action]

In [36]:
simu = simulator()

In [43]:
# Define the Gaussian Process
cov = combo.gp.cov.gauss(X.shape[0])
mean = combo.gp.mean.const()
lik = combo.gp.lik.gauss()
gp = combo.gp.model(lik=lik, mean = mean, cov = cov)

In [45]:
# set the configure for searching 
config = combo.misc.set_config()
config.load('config.ini')

In [ ]:
simu = simulator()
search = combo.search.bayes_policy(simu, X, config)
search.set_seed(0)
search.run( gp )

start the random search .....

001-th step: f(x) = -1.037109, max_f(x) = -1.037109 

002-th step: f(x) = -1.040397, max_f(x) = -1.037109 

003-th step: f(x) = -0.995182, max_f(x) = -0.995182 

004-th step: f(x) = -1.120972, max_f(x) = -0.995182 

005-th step: f(x) = -1.092801, max_f(x) = -0.995182 

006-th step: f(x) = -1.104426, max_f(x) = -0.995182 

007-th step: f(x) = -1.082187, max_f(x) = -0.995182 

008-th step: f(x) = -1.145944, max_f(x) = -0.995182 

009-th step: f(x) = -1.069718, max_f(x) = -0.995182 

010-th step: f(x) = -1.046275, max_f(x) = -0.995182 

011-th step: f(x) = -1.035465, max_f(x) = -0.995182 

012-th step: f(x) = -1.095248, max_f(x) = -0.995182 

013-th step: f(x) = -1.188696, max_f(x) = -0.995182 

014-th step: f(x) = -1.055151, max_f(x) = -0.995182 

015-th step: f(x) = -1.122468, max_f(x) = -0.995182 

016-th step: f(x) = -0.966407, max_f(x) = -0.966407 

017-th step: f(x) = -1.013328, max_f(x) = -0.966407 

018-th step: f(x) = -1.137878, max_f(x) = -0.966407